In [ ]:
import numpy as np
import pandas as pd
import SAGA
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from operator import itemgetter
import os
from dotenv import load_dotenv
import matplotlib.colors as mcolors

from SAGA import ObjectCuts as C

load_dotenv()

In [ ]:
custom_palette = ['#377eb8', '#ff7f00', '#4daf4a','#f781bf', '#a65628', '#984ea3','#999999', '#e41a1c', '#dede00']

deut_pallete = ["#FFEBAE", "#ffe599","#f1e594","#e1e592","#d1e591","#c0e492","#9ae39a","#85e2a0",'#6de0a8',"#50deb1","#1cd2b8","#00c6bd","#00b9c0","#00acc1","#009fbf","#0091ba","#0083b3","#0075a9","#2a679c","#295f97","#295792","#2b4f8d","#2e4787","#313f80","#343679","#372e72","#3a246a"]

custom_style = {
    "figure.facecolor": "#000000",
    "axes.facecolor": "#000000",
    "savefig.facecolor": "#000000", 
    "grid.color": "#000000",
    "text.color": "#ffe599",
    "axes.labelcolor": "#ffe599",
    "axes.edgecolor": "#ffe599",
    "xtick.color": "#ffe599",
    "ytick.color": "#ffe599",
    "grid.linestyle": "-",
    "lines.solid_capstyle": "round",
    "xtick.color": "#ffe599",
    "xtick.major.size": "4",
    "xtick.minor.size": "4",
    "xtick.major.pad": "4",
    "xtick.minor.pad": "4",
    "xtick.direction": "in",
    "xtick.labelsize": "medium",
    "ytick.direction": "in",
    "ytick.labelsize": "medium"
}
new_custom_palette = ['#9fc5e8', "#ffe599", "#29af7fff"]
sns.set_style("ticks", custom_style)



In [ ]:
# Data manipulations

def get_radii(coordinates):
    radii = []
    for coordinate_pair in coordinates:
        radii.append(np.sqrt(coordinate_pair[0]**2+coordinate_pair[1]**2))
    return radii
    
def get_ellipticities(coordinates):
    # # First get quadrupole moments
    coordinates_squared = np.square(coordinates)

    xdiff_ydiff_col = np.prod(coordinates, axis=2)
    xdiff_ydiff_col = xdiff_ydiff_col[:,:,np.newaxis]

    quadrupole_moments_setup = np.insert(coordinates_squared, 0, [[[xdiff_ydiff_col]]], axis=2) # diffx*diffy now at position 0
    quadrupole_moments = quadrupole_moments_setup.mean(axis=1)

    # # Get ellipticity from quadrupole moments
    # Q_xy, Q_xx, Q_yy = np.quadrupole_moments
    quadrupole_xx_yy = quadrupole_moments[:,1:3]
    quadrupole_xy = quadrupole_moments[:,0:1]

    e_1_numerator = -np.diff(quadrupole_xx_yy, axis=1)
    e_2_numerator = 2*quadrupole_xy

    e_denominator_1 = np.sum(quadrupole_xx_yy, axis=1)[:,np.newaxis]
    e_denominator_inside_sqrt = np.prod(quadrupole_xx_yy, axis=1)[:,np.newaxis] - quadrupole_xy**2
    e_denominator = e_denominator_1 + 2*np.sqrt(e_denominator_inside_sqrt)

    e_1 = e_1_numerator / e_denominator
    e_2 = e_2_numerator / e_denominator

    ellipticities = np.sqrt(e_1**2 + e_2**2)[:,0]

    return ellipticities


def get_randomized_ellipticities(radii, num_iterations):
    num = len(radii)
    radial_stdev = np.std(radii)

    # Create "isotropic" (random from uniform - not uniform) angles
    coords = np.random.randn(num_iterations, num, 2)  # (number of times to repeat data set generation, how many satellites, how many spatial dimensions)
    calc_radii = np.sqrt((coords**2).sum(axis=2))
    normalized_coords = coords/calc_radii[:,:,np.newaxis]

    # Move the positions of satellites out or in radially until they reach original radial scale
    reshaped_radii = np.reshape(radii, (1,len(radii), 1))
    scaled_coords = normalized_coords*reshaped_radii

    ellipticities = get_ellipticities(scaled_coords,)
    
    return ellipticities


def get_prominence(actual, radii, num_iterations):
    randomized = get_randomized_ellipticities(radii, num_iterations)
    abv_actual = np.where(randomized >= actual, 1, 0)
    p_above_actual = np.sum(abv_actual)/num_iterations
    prominence = 1/p_above_actual
    
    return prominence

In [ ]:
# Dataframe manipulations
def filter_subhalo_dataframe_by_num_satellites(dataframe, cutoff):
    filtered_dataframe = dataframe[dataframe['vmax_mpeak'] >= cutoff].copy()
    filtered_dataframe = filtered_dataframe.groupby(['HOSTID']).filter(lambda x: len(x) > 2).copy()
    return filtered_dataframe

def add_coordinate_columns(dataframe):
    dataframe['x_coord'] = dataframe['x_adj'] - dataframe['x_host']
    dataframe['y_coord'] = dataframe['y_adj'] - dataframe['y_host']
    # dataframe['z_coord'] = dataframe['z_adj'] - dataframe['z_host']
    # dataframe['3d_coordinates'] = dataframe[['x_coord', 'y_coord', 'z_coord']].values.tolist()
    dataframe['2d_coordinates'] = dataframe[['x_coord', 'y_coord']].values.tolist()

def get_systems_dataframe(dataframe):
    systems_df = dataframe.groupby('HOSTID')['2d_coordinates'].apply(tuple).reset_index(name='coordinates_list')
    systems_df['coordinates_list'] = systems_df['coordinates_list'].tolist()
    systems_df['radii'] = systems_df['coordinates_list'].apply(get_radii)
    systems_df['Dataset'] = 'Actual'
    return systems_df

def add_ellipticity_column(dataframe):
    dataframe['ellipticity'] = dataframe.apply(lambda x: get_ellipticities([x['coordinates_list']])[0], axis=1)

def get_isotropic_data(dataframe):
    dataframe_copy = dataframe.copy()
    dataframe_copy['ellipticity'] = dataframe_copy.apply(lambda x: get_randomized_ellipticities(x['radii'], 1)[0], axis=1)
    dataframe_copy['Dataset'] = 'Isotropic'
    return dataframe_copy

def add_num_subs_column(dataframe):
    dataframe['num_subs'] = dataframe['radii'].str.len()
    display(dataframe)

def add_prominence_column(dataframe, num_iterations):
    dataframe['prominence'] = dataframe.apply(lambda x: get_prominence(x['ellipticity'], x['radii'], num_iterations), axis=1)

def get_prominence_dataframe(dataframe):
    prominence_data = dataframe[['HOSTID', 'ellipticity', 'prominence', 'Dataset']].copy()
    return prominence_data

def plot_prominence(data, num_iterations):
    format = {
        "figure.facecolor": "212946",
        "axes.facecolor": "212946",
        "savefig.facecolor": "212946", 
        "grid.color": "2A3459",
        "text.color": "0.9",
        "axes.labelcolor": "0.9",
        "xtick.color": "0.9",
        "ytick.color": "0.9",
        "grid.linestyle": "-",
        "lines.solid_capstyle": "round",
        "figure.figsize" : "(11.7,8.27)"
    }

    sns.set_style("darkgrid", format)

    fig, ax1 = plt.subplots()
    sp1 = sns.ecdfplot(data=data, x="prominence", hue="Dataset", hue_order=['Actual', 'Isotropic'], complementary=True, ax=ax1)
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xlim(1,num_iterations)
    plt.show()

def plot_prominence_histogram(data, num_iterations):
    format = {
        "figure.facecolor": "212946",
        "axes.facecolor": "212946",
        "savefig.facecolor": "212946", 
        "grid.color": "2A3459",
        "text.color": "0.9",
        "axes.labelcolor": "0.9",
        "xtick.color": "0.9",
        "ytick.color": "0.9",
        "grid.linestyle": "-",
        "lines.solid_capstyle": "round",
        "figure.figsize" : "(11.7,8.27)"
    }

    sns.set_style("darkgrid", format)

    fig, ax1 = plt.subplots()
    sp1 = sns.histplot(data=data, x="prominence", hue="Dataset", hue_order=['Actual', 'Isotropic'], bins=7, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=ax1, palette='gist_rainbow')
    ax1.set_xscale('log')
    # ax1.set_yscale('log')
    # ax1.set_xlim(1,num_iterations)
    plt.show()



In [ ]:
def get_ellipticity_and_prominence_data(dataframe, num_iterations, num_isotropic):
    add_coordinate_columns(dataframe)
    systems_dataframe = get_systems_dataframe(dataframe).copy()
    add_ellipticity_column(systems_dataframe) # this is the original dataframe
    add_prominence_column(systems_dataframe, num_iterations) # bottleneckyy
    prominence_data = get_prominence_dataframe(systems_dataframe).copy()

    for i in range (0,num_isotropic):
        systems_dataframe_isotropic = get_isotropic_data(systems_dataframe).copy()
        add_prominence_column(systems_dataframe_isotropic, num_iterations)  
        prominence_data_isotropic = get_prominence_dataframe(systems_dataframe_isotropic).copy()
        prominence_data = pd.concat([prominence_data, prominence_data_isotropic], ignore_index=True)

    return [systems_dataframe, prominence_data]

## Get Data

In [ ]:
%store -r saga_sats
%store -r VSMDPL_subs_raw
%store -r VSMDPL_subs_interlopers

VSMDPL_subs_raw = VSMDPL_subs_raw.copy()
saga_sats = saga_sats.copy()
VSMDPL_subs_interlopers = VSMDPL_subs_interlopers.copy()

In [ ]:
print(len(VSMDPL_subs_interlopers.interloper_host_id.unique()))
print(len(VSMDPL_subs_raw.upid.unique()))

In [ ]:
HOST_PROPERTIES = os.getenv('HOST_PROPERTIES')
saga_hosts = pd.read_parquet(HOST_PROPERTIES)

iterations = 10000

saga_sats.rename(columns = {'X':'x_adj', 'Y':'y_adj', 'HOST_X':'x_host', 'HOST_Y':'y_host'}, inplace = True)
VSMDPL_subs_raw.rename(columns = {'upid':'HOSTID'}, inplace = True)
VSMDPL_subs_interlopers.rename(columns = {'interloper_host_id':'HOSTID', 'interloper_host_x':'x_host', 'interloper_host_y':'y_host', 'interloper_host_vmax_mpeak':'host_vmax_mpeak'}, inplace = True)

# Use vmax_mpeak as proxy for luminosity
vmax_cutoff_for_avg_num = {
    '3.5' : 61,
      '4' : 51.6,
    '4.5' : 46.4,
      '5' : 43.5,
    '5.5' : 40.98,
    'saga': 44.09, # 4.855 sats per host on average
    'saga_interlopers': 46.77 # 4.855 sats per host on average
}

VSMDPL_subs_multi = {}
for cutoff in list(vmax_cutoff_for_avg_num.keys())[:5]:
    VSMDPL_subs_multi[cutoff] = filter_subhalo_dataframe_by_num_satellites(VSMDPL_subs_raw, vmax_cutoff_for_avg_num[cutoff])

VSMDPL_subs = filter_subhalo_dataframe_by_num_satellites(VSMDPL_subs_raw, vmax_cutoff_for_avg_num['saga'])
VSMDPL_subs_interlopers = filter_subhalo_dataframe_by_num_satellites(VSMDPL_subs_interlopers, vmax_cutoff_for_avg_num['saga_interlopers'])

saga_sats = saga_sats.groupby(['HOSTID']).filter(lambda x: len(x) > 2).copy()

## SAGA Analysis

In [ ]:
sns.set_style("ticks", custom_style)

saga_data, saga_plot = get_ellipticity_and_prominence_data(saga_sats, iterations, 1)
plot_prominence(saga_plot, iterations)

#### Generate 100x isotropic data

In [ ]:
saga_data, saga_plot = get_ellipticity_and_prominence_data(saga_sats, iterations, 100)
plot_prominence(saga_plot, iterations)

## SIM Analysis

In [ ]:
VSMDPL_data, VSMDPL_plot = get_ellipticity_and_prominence_data(VSMDPL_subs, iterations, 1)
plot_prominence(VSMDPL_plot, iterations)

## SIM Interloper Analysis

In [ ]:
VSMDPL_interloper_data, VSMDPL_interloper_plot = get_ellipticity_and_prominence_data(VSMDPL_subs_interlopers, iterations, 1)
plot_prominence(VSMDPL_interloper_plot, iterations)
VSMDPL_interloper_data

## SAGA + Sim (Halofinder)

In [ ]:
saga_plot['Datasource'] = 'Saga'
VSMDPL_plot['Datasource'] = 'VSMDPL (RS)'
saga_sim_data = pd.concat([saga_plot, VSMDPL_plot], ignore_index=True)

fig, ax1 = plt.subplots()
sp1 = sns.ecdfplot(data=saga_sim_data, x="prominence", hue=saga_sim_data[['Datasource', 'Dataset']].apply(tuple, axis=1), complementary=True, ax=ax1)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlim(1,10000)

lss = [':', '--', '-.', '-']

handles = sp1.legend_.legendHandles[::-1]

for line, ls, handle in zip(sp1.lines, lss, handles):
    line.set_linestyle(ls)
    handle.set_ls(ls)


fig.set_size_inches([6, 4])
fig.savefig("figures/SAGAvVSMDPLProminence.png",dpi=600)

In [ ]:
random_normal_df = pd.DataFrame(list(np.arange(1,1000001,1)), columns=['ordered_list'])
random_normal_df['prominence'] = random_normal_df.size/(random_normal_df.size - random_normal_df.iloc[lambda x: x.index]+1)
random_normal_df['Datasource'] = 'Isotropic'
random_normal_df

### SAGA + Sim (interlopers)

In [ ]:
sns.set_style("ticks", custom_style)

# random = pd.DataFrame(list(prominence_arr), columns=['prominence'])
random_normal_df['Dataset'] = 'Actual'
saga_plot['Datasource'] = 'Saga'
VSMDPL_interloper_plot['Datasource'] = 'VSMDPL'
saga_sim_data = pd.concat([saga_plot, VSMDPL_interloper_plot, random_normal_df], ignore_index=True)

fig, ax1 = plt.subplots()
sp1 = sns.ecdfplot(data=saga_sim_data[saga_sim_data['Dataset'] == 'Actual'], x="prominence", hue=saga_sim_data['Datasource'], complementary=True, ax=ax1, palette=new_custom_palette)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlim(1,10000)
ax1.set_ylim(0.0001,1)

saga_lim = ax1.axhline(y=1/len(list(saga_plot.HOSTID.unique())), color=new_custom_palette[0], linestyle='-', alpha=0.5)
VSMDPL_lim = ax1.axhline(y=1/len(list(VSMDPL_interloper_plot.HOSTID.unique())), color=new_custom_palette[1], linestyle='-', alpha=0.5)

lss = [':', '-.', '-']

handles = sp1.legend_.legendHandles[::-1]
ax1.legend_.set_title('')
ax1.set_xlabel('Prominence $\mathcal{P}_e$')
ax1.set_ylabel('Fraction > $\mathcal{P}_e$')
sp1.set_xticks([1, 10, 100, 1000, 10000])
sp1.tick_params(which='both', direction="in")

for line, ls, handle in zip(sp1.lines, lss, handles):
    line.set_linestyle(ls)
    handle.set_ls(ls)

fig.set_size_inches([5, 4])
fig.savefig("figures/SAGAvVSMDPLProminence.png",dpi=600)

## Sim (Halofinder) + Sim (Interlopers)

In [ ]:
sim_sim_data = pd.concat([VSMDPL_plot, VSMDPL_interloper_plot], ignore_index=True)

fig, ax1 = plt.subplots()
sp1 = sns.ecdfplot(data=sim_sim_data[sim_sim_data['Dataset'] == 'Actual'], x="prominence", hue=sim_sim_data['Datasource'], complementary=True, ax=ax1)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlim(1,10000)

lss = [':', '--', '-.', '-']

handles = sp1.legend_.legendHandles[::-1]

for line, ls, handle in zip(sp1.lines, lss, handles):
    line.set_linestyle(ls)
    handle.set_ls(ls)


fig.set_size_inches([6, 4])
fig.savefig("figures/VSMDPLvVSMDPLProminence.png",dpi=600)

## Saga + Sim + sim

In [ ]:
sns.set_style("white")

saga_sim_sim_data = pd.concat([saga_plot, sim_sim_data], ignore_index=True)

fig, ax1 = plt.subplots()
sp1 = sns.ecdfplot(data=saga_sim_sim_data[saga_sim_sim_data['Dataset'] == 'Actual'], x="prominence", hue=saga_sim_sim_data['Datasource'], complementary=True, ax=ax1)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlim(1,10000)
ax1.set_xlabel('Prominence $\mathcal{P}_e$')
ax1.set_ylabel('Density')


lss = [':', '--', '-.', '-']

handles = sp1.legend_.legend_handles[::-1]
ax1.legend_.set_title('')

for line, ls, handle in zip(sp1.lines, lss, handles):
    line.set_linestyle(ls)
    handle.set_ls(ls)

fig.set_size_inches([8, 5])
fig.savefig("figures/VSMDPLvVSMDPLProminence.png",dpi=600)





fig2, ax2 = plt.subplots()
sp2 = sns.histplot(data=saga_sim_sim_data[saga_sim_sim_data['Dataset'] == 'Actual'], x="prominence", hue="Datasource", bins=50, element="poly", stat="percent", common_norm=False, common_bins=True, fill=True, ax=ax2, palette='gist_rainbow', log_scale=[True,True])

lss = ['--', '-', '-.']

handles = ax2.legend_.legendHandles[::-1]
ax2.legend_.set_title('')

ax2.set_xlim(1,10000)

for line, ls, handle in zip(sp1.lines, lss, handles):
    line.set_linestyle(ls)
    handle.set_ls(ls)

fig2.set_size_inches([8, 5])


### Ellipticity SAGA + Sim (Halofinder)

In [ ]:
sns.set_style("ticks", custom_style)

saga_data['Datasource'] = 'Saga'
VSMDPL_data['Datasource'] = 'VSMDPL (RS)'
saga_sim_ellipticity_data = pd.concat([saga_data, VSMDPL_data], ignore_index=True)

fig, ax1 = plt.subplots()
ax1.set_xlabel('$\longleftarrow$ more spherical         Ellipticity (2D)         more elliptical $\longrightarrow$')
sp1 = sns.histplot(saga_sim_ellipticity_data, x="ellipticity", hue="Datasource", hue_order=['Saga', 'VSMDPL (RS)'], bins=7, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=ax1, palette='gist_rainbow')

lss = ['--', '-']

handles = sp1.legend_.legendHandles[::-1]

for line, ls, handle in zip(sp1.lines, lss, handles):
    line.set_linestyle(ls)
    handle.set_ls(ls)


fig.set_size_inches([6, 4])
fig.savefig("figures/SAGAvVSMDPLEllipticity.png",dpi=600)

### Ellipticity SAGA + Sim (Interlopers)

In [ ]:
sns.set_style("ticks", custom_style)


saga_data['Datasource'] = 'Saga'
VSMDPL_interloper_data['Datasource'] = 'VSMDPL'
saga_sim_ellipticity_data = pd.concat([saga_data, VSMDPL_interloper_data], ignore_index=True)

fig, ax1 = plt.subplots()
ax1.set_xlabel('$\longleftarrow$ circular              Ellipticity (2D)                 linear $\longrightarrow$')
sp1 = sns.histplot(saga_sim_ellipticity_data, x="ellipticity", hue="Datasource", hue_order=['Saga'], bins=11, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=ax1, palette=['#9fc5e8', "#ffe599"])
sp2 = sns.histplot(saga_sim_ellipticity_data, x="ellipticity", hue="Datasource", hue_order=['VSMDPL'], bins=20, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=ax1, palette=['#ffe599', "#9fc5e8"])
sp1.set_xticks([0, .2, .4, .6, .8, 1])
sp1.set_yticks([0, .4, .8, 1.2, 1.6, 2.0])
sp1.set_yticklabels(['', .4, .8, 1.2, 1.6, 2.0])
sp1.tick_params(which='both', direction="in")

lss = ['--', '-']

handles = sp1.legend_.legendHandles[::-1]


for line, ls, handle in zip(sp2.lines, lss, handles):
    line.set_linestyle(ls)
    handle.set_ls(ls)

ax1.legend_.set_title('')
ax1.legend(title='', loc='upper right', labels=['SAGA', 'VSMDPL'])


fig.set_size_inches([5, 4])
fig.savefig("figures/SAGAvVSMDPLEllipticity.png",dpi=600)

In [ ]:
print("Saga mean ellipticity: {}".format(saga_data.ellipticity.mean()))
print("Sim mean ellipticity: {}".format(VSMDPL_interloper_data.ellipticity.mean()))

## Ellipticity Sim + Sim (plus prominence for fig)

In [ ]:
sns.set_style("ticks", custom_style)

## Ellipticity Plot
VSMDPL_interloper_data['Datasource'] = 'VSMDPL (Int)'
VSMDPL_data['Datasource'] = 'VSMDPL (RS)'
saga_sim_ellipticity_data = pd.concat([VSMDPL_data, VSMDPL_interloper_data], ignore_index=True)

fig, axs = plt.subplots(ncols=2, layout="constrained")
axs[0].set_xlabel('$ \longleftarrow$ circular   Ellipticity (2D)   linear $\longrightarrow$ ')
sp1 = sns.histplot(saga_sim_ellipticity_data, x="ellipticity", hue="Datasource", hue_order=['VSMDPL (RS)', 'VSMDPL (Int)'], bins=7, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=axs[0], palette=['#9fc5e8', "#ffe599"])
 

lss = ['--', '-']

handles = ax1.legend_.legendHandles[::-1]
axs[0].legend_.set_title('')
axs[0].tick_params(which='both', direction="in")
axs[0].set_yticks([0, .25, .5, .75, 1.0, 1.25, 1.5, 1.75, 2.0])
axs[0].set_yticklabels(['', .25, .50, .75, 1.00, 1.25, 1.50, 1.75, 2.00])
axs[0].set_xticks([0, .2, .4, 0.6, 0.8, 1])

for line, ls, handle in zip(sp1.lines, lss, handles):
    line.set_linestyle(ls)
    handle.set_ls(ls)


## Prominence Plot
sns.set_style("white")
VSMDPL_interloper_plot['Datasource'] = 'VSMDPL (Int)'
sim_sim_data = pd.concat([VSMDPL_plot, VSMDPL_interloper_plot], ignore_index=True)

sp2 = sns.ecdfplot(data=sim_sim_data[sim_sim_data['Dataset'] == 'Actual'], x="prominence", hue=sim_sim_data['Datasource'], complementary=True, ax=axs[1], palette=['#9fc5e8', "#ffe599"])
axs[1].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xlim(1,10000)
axs[1].set_xlabel('Prominence $\mathcal{P}_e$')
axs[1].set_ylabel('Proportion', labelpad=0)
axs[1].legend_.set_title('')
axs[1].tick_params(which='both', direction="in")



lss = ['--', '-']

handles = sp2.legend_.legendHandles[::-1]

for line, ls, handle in zip(sp2.lines, lss, handles):
    line.set_linestyle(ls)
    handle.set_ls(ls)


fig.set_size_inches([7, 3])
fig.savefig("figures/VSMDPLvVSMDPLProminenceAndEllipticity.png",dpi=600)

## Ellipticity SAGA + Sim rockstar + Sim interlopers

In [ ]:
VSMDPL_interloper_data['Datasource'] = 'VSMDPL (Int)'
saga_data['Datasource'] = 'SAGA'
saga_sim_ellipticity_data = pd.concat([saga_data, VSMDPL_data, VSMDPL_interloper_data], ignore_index=True)

fig, ax1 = plt.subplots()
ax1.set_xlabel('$\longleftarrow$ more spherical         Ellipticity (2D)         more elliptical $\longrightarrow$')
sp1 = sns.histplot(saga_sim_ellipticity_data, x="ellipticity", hue="Datasource", hue_order=['SAGA', 'VSMDPL (RS)', 'VSMDPL (Int)'], bins=7, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=ax1)
 

lss = ['--', '-', '-.']

handles = ax1.legend_.legendHandles[::-1]
ax1.legend_.set_title('')

for line, ls, handle in zip(sp1.lines, lss, handles):
    line.set_linestyle(ls)
    handle.set_ls(ls)


fig.set_size_inches([5, 4])
fig.savefig("figures/SAGAvVSMDPLvInterloperEllipticity.png",dpi=600)

## SIM Compare prominence for different avg # satellites

In [ ]:
VSMDPL_systems_multi = {}
VSMDPL_plot_multi = {}

for avg in list(vmax_cutoff_for_avg_num.keys())[:5]:
    VSMDPL_systems_multi[avg], VSMDPL_plot_multi[avg] = get_ellipticity_and_prominence_data(VSMDPL_subs_multi[avg], iterations, 1)
    VSMDPL_plot_multi[avg]['Average # Satellites'] = avg
    VSMDPL_systems_multi[avg]['Datasource'] = 'VSMDPL ' + avg

In [ ]:
VSMDPL_multi_plot_data = pd.concat(VSMDPL_plot_multi, ignore_index=True)
VSMDPL_multi_plot_data = VSMDPL_multi_plot_data[VSMDPL_multi_plot_data['Dataset'] == 'Actual'].copy()

fig, ax1 = plt.subplots()
sp1 = sns.ecdfplot(data=VSMDPL_multi_plot_data, x="prominence", hue='Average # Satellites', hue_order=['3.5', '4', '4.5', '5', '5.5'], complementary=True, ax=ax1)
ax1.set_xscale('log')
ax1.set_xlim(1,iterations)

In [ ]:
for df in VSMDPL_systems_multi.values():
    add_num_subs_column(df)

In [ ]:
sns.set_style("ticks", custom_style)

VSMDPL_multi_data = pd.concat(VSMDPL_systems_multi, ignore_index=True)
fig, ax1 = plt.subplots()
ax1.set_xlabel('$\longleftarrow$ circular                  Ellipticity (2D)                       linear $\longrightarrow$')
sp1 = sns.histplot(VSMDPL_multi_data, x="ellipticity", hue="Datasource", hue_order=['VSMDPL 3.5', 'VSMDPL 4', 'VSMDPL 4.5', 'VSMDPL 5', 'VSMDPL 5.5'], bins=7, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=ax1, palette='viridis')

In [ ]:
all_data = pd.concat([VSMDPL_multi_data, saga_sim_ellipticity_data], ignore_index=True)

fig, ax1 = plt.subplots()
ax1.set_xlabel('$\longleftarrow$ circular              Ellipticity (2D)                 linear $\longrightarrow$')
sp1 = sns.histplot(all_data, x="ellipticity", hue="Datasource", hue_order=['Saga', 'VSMDPL', 'VSMDPL 3.5', 'VSMDPL 4', 'VSMDPL 4.5', 'VSMDPL 5', 'VSMDPL 5.5'], bins=7, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=ax1, palette=custom_palette)


fig.set_size_inches([7, 3])


fig.savefig("figures/VSMDPLNumSatellitesEllipticity.png",dpi=600)

### VSMDPL Interloper cuts

In [ ]:
VSMDPL_interloper_data, VSMDPL_interloper_plot = get_ellipticity_and_prominence_data(VSMDPL_subs_interlopers, iterations, 1)
plot_prominence(VSMDPL_interloper_plot, iterations)

## Ellipticity and Prominence vs Host properties

In [ ]:
add_num_subs_column(saga_data)
saga = saga_hosts.merge(saga_data, on="HOSTID")
saga = saga.rename(columns={"num_subs":"num_sats"})
saga_hosts

In [ ]:
saga['num_sats'] = saga['num_sats'].astype(np.float64)
saga.sort_values('prominence')

In [ ]:
sns.set_style("ticks", custom_style)

sns.scatterplot(saga, x='ellipticity', y='prominence', edgecolor=None, hue=saga['num_sats'], palette='cividis', legend='full')
plt.yscale('log')
plt.xlim([0,1])
plt.show()

In [ ]:
isotropic_saga_data = saga_plot[saga_plot['Dataset']=='Isotropic'].copy()
isotropic_saga_data = isotropic_saga_data.merge(saga_data[['HOSTID', 'radii']], on='HOSTID').copy()
add_num_subs_column(isotropic_saga_data)
isotropic_saga_data = isotropic_saga_data.rename(columns={"num_subs":"num_sats"})


sns.scatterplot(isotropic_saga_data, x='ellipticity', y='prominence', edgecolor=None, hue=isotropic_saga_data['num_sats'], palette='viridis', legend='full', size=1)
plt.yscale('log')
plt.xlim([0,1])
plt.show()

In [ ]:
saga_copy = saga[['HOSTID', 'ellipticity', 'prominence', 'Dataset', 'radii', 'num_sats']]
raw_and_isotropic_saga_data = pd.concat([saga_copy, isotropic_saga_data])
raw_and_isotropic_saga_data.loc[raw_and_isotropic_saga_data["Dataset"] == "Actual", "edgecolor"] = 'white'
raw_and_isotropic_saga_data.loc[raw_and_isotropic_saga_data["Dataset"] == "Isotropic", "edgecolor"] = 'None'
raw_and_isotropic_saga_data


In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(raw_and_isotropic_saga_data[raw_and_isotropic_saga_data['Dataset']=='Isotropic'], x='ellipticity', y='prominence', hue='num_sats', edgecolor=None, palette='viridis', legend=None, ax=ax, size=0.1)
sns.scatterplot(raw_and_isotropic_saga_data[raw_and_isotropic_saga_data['Dataset']=='Actual'], x='ellipticity', y='prominence', hue='num_sats', edgecolor='white', palette='viridis', legend=None, ax=ax, linewidth=1)
plt.yscale('log')
plt.xlim([0,1])
plt.show()

In [ ]:
fig, axs = plt.subplots(ncols=4)
fig.set_size_inches(30, 6, forward=True)
sns.scatterplot(data=saga, x="MORPHTYPE", y="ellipticity", edgecolor=None, ax=axs[0])
sns.scatterplot(data=saga, x="radius", y="ellipticity", edgecolor=None, ax=axs[1])
sns.scatterplot(data=saga, x="ba", y="ellipticity", edgecolor=None, ax=axs[2])
sns.scatterplot(data=saga, x="phi", y="ellipticity", edgecolor=None, ax=axs[3])

In [ ]:
fig, axs = plt.subplots(ncols=4)
fig.set_size_inches(30, 6, forward=True)
sns.scatterplot(data=saga, x="sb_r", y="ellipticity", ax=axs[0])
sns.scatterplot(data=saga, x="SERSIC", y="ellipticity", ax=axs[1])
sns.scatterplot(data=saga, x="K_ABS", y="ellipticity", ax=axs[2])
sns.scatterplot(data=saga, x="gr", y="ellipticity", ax=axs[3])

In [ ]:
spearmanrs = {}
props = ["MORPHTYPE", "radius", "ba", "phi", "sb_r", "SERSIC", "K_ABS", "gr", "num_sats"]
for prop in props:
    spearmanr_results = sp.stats.spearmanr(saga['prominence'], saga[prop], axis=0, nan_policy='propagate', alternative='two-sided')
    spearmanrs[prop] = spearmanr_results

In [ ]:
sp.stats.spearmanr(np.array(saga['phi']), np.array(saga['prominence']))

In [ ]:
fig,ax=plt.subplots()
plt.scatter(x=np.array(saga['phi']), y=np.array(saga['prominence']))
print(sp.stats.spearmanr(np.array(saga['phi']), np.array(saga['prominence'])))

In [ ]:
fig, axs = plt.subplots(ncols=4)
fig.set_size_inches(30, 6, forward=True)
for i in range (0,4):
    statistic = spearmanrs[props[i]].statistic.round(2)
    pvalue = spearmanrs[props[i]].pvalue.round(2)
    spearman_string = "Coefficient: " + str(statistic) + " P-value: " + str(pvalue)
    sns.scatterplot(data=saga, x=props[i], y="prominence", ax=axs[i])
    axs[i-4].text(0.99, 0.99, spearman_string, ha='right', va='top', size=20, transform = axs[i].transAxes)
axs[1].set_xscale('log') # just for radius
for ax in axs:
    ax.set_yscale('log')

In [ ]:
fig, axs = plt.subplots(ncols=4)
fig.set_size_inches(30, 6, forward=True)
for i in range (4,8):
    statistic = spearmanrs[props[i]].statistic.round(2)
    pvalue = spearmanrs[props[i]].pvalue.round(2)
    spearman_string = "Coefficient: " + str(statistic) + " P-value: " + str(pvalue)
    sns.scatterplot(data=saga, x=props[i], y="prominence", ax=axs[i-4])
    axs[i-4].text(0.99, 0.99, spearman_string,
        ha='right',
        va='top',
        size=20,
        transform = axs[i-4].transAxes)
for ax in axs:
    ax.set_yscale('log')

In [ ]:
# sns.set_style('ticks')
sns.set_style("ticks", custom_style)

fig, axs = plt.subplots(ncols=2, sharey=True)
fig.set_size_inches(8, 3, forward=True)

figs = [1, 6]
fig_labels = ['$r_{host}$ (kpc)', '$K_{abs}$']

for i in figs: # update w/ enumerate
    i_index = figs.index(i)
    statistic = spearmanrs[props[i]].statistic.round(2)
    pvalue = spearmanrs[props[i]].pvalue.round(2)
    spearman_string = '$\\rho_S$: ' + str(statistic) + ", P: " + str(pvalue)
    sns.scatterplot(data=saga, x=props[i], y="prominence", ax=axs[i_index])
    axs[i_index].text(0.8, 1.14, spearman_string,
        ha='right',
        va='top',
        size=12,
        transform = axs[i_index].transAxes)
    axs[i_index].set_xlabel(fig_labels[i_index], fontsize=14)
for ax in axs:
    ax.set_yscale('log')
axs[0].set_ylabel('Prominence $\mathcal{P}_e$', fontsize=12)
# axs[0].set_xticks([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])

fig.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0)
fig.savefig("figures/ProminenceVProperties.png",dpi=1000)

## Highest Luminosity satellite per system

In [ ]:
highest_luminosity_sats = saga_sats.groupby('HOSTID', as_index = False)['Mr'].min()
highest_luminosity_prominence = pd.merge(highest_luminosity_sats, saga, how="left", left_on="HOSTID", right_on="HOSTID")
highest_luminosity_prominence_SAGA = highest_luminosity_prominence[~highest_luminosity_prominence['prominence'].isna()]

In [ ]:
import scipy as sp
spearmanr_results_lum = sp.stats.spearmanr(highest_luminosity_prominence_SAGA['prominence'], highest_luminosity_prominence_SAGA['Mr'], axis=0, nan_policy='propagate', alternative='two-sided')
spearman_string_lum = "Coefficient: " + str(spearmanr_results_lum.statistic.round(2)) + " P-value: " + str(spearmanr_results_lum.pvalue.round(2))

fig, ax = plt.subplots()
sns.scatterplot(data=highest_luminosity_prominence_SAGA, x='Mr', y='prominence', ax=ax)
ax.set_yscale('log')
plt.text(0.99, 0.99, spearman_string_lum, ha='right', va='top', size=15, transform = ax.transAxes)

In [ ]:
host_property='num_sats'

fig, axs = plt.subplots()
spearmanr_results = sp.stats.spearmanr(saga['prominence'], saga[host_property], axis=0, nan_policy='propagate', alternative='less')
display(spearmanr_results)

statistic = spearmanr_results.statistic.round(2)
pvalue = spearmanr_results.pvalue.round(2)
spearman_string = "Coefficient: " + str(statistic) + " P-value: " + str(pvalue)

sns.scatterplot(data=saga, x=host_property, y="prominence", ax=axs)
axs.set_yscale('log') # just for radius
plt.text(2, 2, spearman_string, ha='right', va='top', size=15, transform = ax.transAxes)
axs.set_xlabel('# satellites')
axs.set_ylabel('$\mathcal{P}_e$')
axs.set_xticks(np.arange(2, 14, 1))


In [ ]:
host_property='phi'
saga[host_property]

fig, axs = plt.subplots()
spearmanr_results = sp.stats.spearmanr(saga['prominence'], saga[host_property], axis=0, nan_policy='propagate', alternative='less')
display(spearmanr_results)

statistic = spearmanr_results.statistic.round(2)
pvalue = spearmanr_results.pvalue.round(2)
spearman_string = "Coefficient: " + str(statistic) + " P-value: " + str(pvalue)

sns.scatterplot(data=saga, x=host_property, y="prominence", ax=axs)
# axs.set_xscale('log') # just for radius
axs.set_yscale('log') # just for radius


In [ ]:
host_property='MORPHTYPE'

fig, axs = plt.subplots()
spearmanr_results = sp.stats.spearmanr(saga['prominence'], saga[host_property], axis=0, nan_policy='propagate', alternative='less')
display(spearmanr_results)

statistic = spearmanr_results.statistic.round(2)
pvalue = spearmanr_results.pvalue.round(2)
spearman_string = "Coefficient: " + str(statistic) + " P-value: " + str(pvalue)

sns.scatterplot(data=saga, x=host_property, y="prominence", ax=axs)
# axs.set_xscale('log') # just for radius
axs.set_yscale('log') # just for radius

fig.set_size_inches([10, 7])
